In [69]:
import pandas as pd
import numpy as np
pd.set_option("max_columns", 50)

diab_final = pd.read_csv("data/clean_data_2class.csv")

# columns need to be deleted since they are not adding any value
del_cols = ["Unnamed: 0", "encounter_id", "patient_nbr", "weight", "discharge_disposition", 
            "admission_source", "payer_code", "medical_specialty", "admisison_type"]

for col in del_cols:
    if col in diab_final.columns:
        diab_final.drop(col, inplace=True, axis = 1)
        #del diab_final[col]

# for stratefied sampling of data
from sklearn.model_selection import train_test_split

# for alarm sound
import winsound

In [70]:
diab_final.shape

(98052, 43)

In [2]:
diab_final.head()

,race,gender,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,admission_type
0,Caucasian,Female,1,3,59,0,18,0,0,0,ENMI,ENMI,ENMI,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,YES,Physician Referral
1,AfricanAmerican,Female,2,2,11,5,13,2,0,1,PCC,ENMI,CFI,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO,Physician Referral
2,Caucasian,Male,3,2,44,1,16,0,0,0,IPD,ENMI,CSD,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO,Physician Referral
3,Caucasian,Male,4,1,51,0,8,0,0,0,NP,NP,ENMI,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO,Physician Referral
4,Caucasian,Male,5,3,31,6,16,0,0,0,CSD,CSD,ENMI,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,YES,Clinic Referral


In [ ]:
# list selected from feature importance from random forest
pick_list = ["number_inpatient", "num_medications", "number_emergency", "number_outpatient", "num_lab_procedures", 
             "number_diagnoses", "age", "time_in_hospital", "num_procedures", "diag_1_NP", "admission_type_HMO Referral", 
             "diabetesMed_Yes", "admission_type_Transfer from another health care facility", "admission_type_Physician Referral", 
             "gender_Male", "diag_2_CSD", "metformin_No", "race_Caucasian", "gender_Female", "insulin_Steady", "insulin_No", 
             "metformin_Steady", "diag_1_ENMI", "diag_3_ENMI", "A1Cresult_None", "admission_type_Clinic Referral", "diag_1_CSD", 
             "diag_3_CSD", "diag_2_ENMI", "diabetesMed_No", "insulin_Down", "change_Ch", "diag_1_PCC", "race_AfricanAmerican", 
             "change_No", "max_glu_serum_None", "diag_2_RSD", "diag_2_SSD"]

In [71]:
# split predictors and outcome
predictors = diab_final.drop("readmitted", axis = 1)
outcome = diab_final["readmitted"]
dummy_predictor = pd.get_dummies(predictors)


In [72]:
dummy_predictor.shape

(98052, 160)

In [73]:
dummy_predictor = dummy_predictor[pick_list]

In [74]:
dummy_predictor.shape

(98052, 38)

In [5]:
# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier

In [38]:
num_trees = range(10,20)
depth_range = range(9,15)
tuning_parameter = []

prev_error = 1
test_size_range = np.arange(.4,0,-0.05)
for test_size in test_size_range:
    X_train, X_test, Y_train, Y_test = train_test_split(dummy_predictor,
                                                   outcome,
                                                   stratify=outcome,
                                                   test_size=test_size,
                                                   random_state = 0)
    for num in num_trees:
        for depth in depth_range:
            clf = RandomForestClassifier(n_estimators = num, n_jobs=-1, random_state=0,
                             max_depth = depth, verbose=0)
            clf.fit(X_train, Y_train)
            rndm_pred = clf.predict(X_test)
#         cm = confusion_matrix(Y_test, rndm_pred)
#         clf_rate = (cm[0,0]+cm[1,1])/sum(sum(cm))
            clf_test = clf.score(X_test,Y_test)
            clf_train = clf.score(X_train,Y_train)
            tuning_parameter.append([1 - test_size, num, depth, clf_train, clf_test])
#             print("Tree count:%d Tree depth:%d, Prediction Accuracy:%.2f" % (num,depth, clf_rate*100))
        
# 

In [53]:
# for 1000 estimators
X_train, X_test, Y_train, Y_test = train_test_split(dummy_predictor,
                                                   outcome,
                                                   stratify=outcome,
                                                   test_size=0.2,
                                                   random_state = 0)

clf = RandomForestClassifier(n_estimators = 1000, n_jobs=-1, random_state=0,
                             max_depth = 200, verbose=0)
clf.fit(X_train, Y_train)
# rndm_pred = clf.predict(X_test)
# cm = confusion_matrix(Y_test, rndm_pred)
# clf_rate = (cm[0,0]+cm[1,1])/sum(sum(cm))
clf_test = clf.score(X_test,Y_test)
clf_train = clf.score(X_train,Y_train)
large_tree = [0.8, 1000, 11, clf_train, clf_test]

In [54]:
large_tree

[0.8, 1000, 11, 0.99998725156487045, 0.61516495844169083]

In [19]:
# tuning_list = [''.join(str(tuning_parameter[i]))[1:-1] for i in range(len(tuning_parameter)) ]

In [58]:
tuning_df_limited = pd.DataFrame(tuning_parameter)
tuning_df_full.columns = ["training_size", "number of trees", "tree depth", "training efficiency", "testing efficiency"]

In [64]:
tuning_df_limited.columns = ["training_size", "number of trees", "tree depth", "training efficiency", "testing efficiency"]

In [33]:
tuning_df_full = pd.DataFrame(tuning_parameter)

In [57]:
tuning_df_full

,0,1,2,3,4
0,0.60,10,9,0.638269,0.618113
1,0.60,10,10,0.646309,0.616201
2,0.60,10,11,0.654893,0.616455
3,0.60,10,12,0.665041,0.616863
4,0.60,10,13,0.681188,0.616328
5,0.60,10,14,0.697795,0.614798
6,0.60,11,9,0.637691,0.618189
7,0.60,11,10,0.645935,0.616761
8,0.60,11,11,0.654111,0.616787
9,0.60,11,12,0.664803,0.616430


In [65]:
tuning_df_full.to_csv("data/RF_full_model.csv")

In [66]:
tuning_df_limited.to_csv("data/RF_limited.csv")

In [67]:
max(tuning_df_limited['testing efficiency'])

0.6255353864980624

In [68]:
max(tuning_df_full['testing efficiency'])

0.62227207831939624

In [42]:
from sklearn.feature_selection import SelectFromModel
sfm = SelectFromModel(clf, threshold=0.15)

In [43]:
feature_importances = pd.DataFrame(clf.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)

In [44]:
feature_importances[feature_importances.importance>0.005]

,importance
number_inpatient,0.194017
num_lab_procedures,0.095148
num_medications,0.089053
time_in_hospital,0.060125
age,0.058983
number_diagnoses,0.057859
number_emergency,0.050489
number_outpatient,0.046167
num_procedures,0.043664
diag_3_CSD,0.014201
